In [249]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import Ridge, Lasso

# Carregar o dataset
df = pd.read_csv("C:/github/Rio-Housing-Scrapper/Data(29-09_2023)/df_cleaned_final.csv")

# Excluir as colunas "Rent" e "Address"
df = df.drop(columns=["Rent", "Address", "Region"])

# Exibir as primeiras linhas do dataframe
df.head()

,Property Type,Total Rent,Area,Rooms,Bathrooms,Garage,Furnished
0,Apartamento,1368,55,2,1,1.0,0
1,Apartamento,6012,140,5,2,1.0,1
2,Apartamento,3198,53,1,1,0.0,0
3,Apartamento,4075,61,2,1,0.0,0
4,Apartamento,2649,27,1,1,0.0,0


In [250]:
# Definir variáveis independentes (features) e a variável dependente (target)
X = df.drop("Total Rent", axis=1)
y = df["Total Rent"]

# Dividir os dados em conjuntos de treino e teste (75% treino, 25% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

# Exibir o tamanho dos conjuntos de treino e teste
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1929, 6), (643, 6), (1929,), (643,))

In [251]:
# Aplicar codificação one-hot à coluna 'Property Type'
X_train = pd.get_dummies(X_train, columns=['Property Type'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['Property Type'], drop_first=True)

# Treinar o modelo de regressão linear novamente
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Prever valores no conjunto de teste e calcular MSE
y_pred = linear_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)


MSE: 4222621.87842033


In [252]:
# Treinar o modelo de Regressão Ridge
ridge_model = Ridge()
ridge_model.fit(X_train, y_train)

# Prever valores no conjunto de teste e calcular MSE para Regressão Ridge
y_pred_ridge = ridge_model.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)

# Treinar o modelo de Regressão Lasso
lasso_model = Lasso()
lasso_model.fit(X_train, y_train)

# Prever valores no conjunto de teste e calcular MSE para Regressão Lasso
y_pred_lasso = lasso_model.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)

mse_ridge, mse_lasso

(4223666.3499856815, 4224117.062425409)

Tratar agora considerando região

In [253]:
# Carregar o dataset
df = pd.read_csv("C:/github/Rio-Housing-Scrapper/Data(29-09_2023)/df_cleaned_final.csv")

# Excluir as colunas "Rent" e "Address"
df = df.drop(columns=["Rent", "Address"])

# Definir variáveis independentes (features) e a variável dependente (target)
X = df.drop("Total Rent", axis=1)
y = df["Total Rent"]

# Dividir os dados em conjuntos de treino e teste (75% treino, 25% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)

# Exibir o tamanho dos conjuntos de treino e teste
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1929, 7), (643, 7), (1929,), (643,))

In [254]:
# Calcular as médias da variável resposta para cada categoria de "Region" usando o conjunto de treino
region_target_mean = y_train.groupby(X_train['Region']).mean()

region_target_mean


Region
Abolição           1317.916667
Andaraí            3283.625000
Anil               2998.709677
Barra da Tijuca    9023.945455
Benfica            1980.500000
                      ...     
Vila Valqueire     2251.076923
Vila da Penha      1912.722222
Vista Alegre       1612.714286
Váz Lobo           1218.555556
Água Santa         1212.000000
Name: Total Rent, Length: 113, dtype: float64

In [255]:
# Substituir cada categoria em "Region" pela média correspondente no conjunto de treino e teste
X_train['Region'] = X_train['Region'].map(region_target_mean)
X_test['Region'] = X_test['Region'].map(region_target_mean)

# Preencher quaisquer valores NaN no conjunto de teste com a média global do conjunto de treino (para regiões não vistas no treino)
X_test['Region'].fillna(y_train.mean(), inplace=True)

# Verificar as primeiras linhas dos conjuntos de treino e teste após o Target Encoding
X_train.head(), X_test.head()

# Aplicar codificação one-hot à coluna 'Property Type'
X_train = pd.get_dummies(X_train, columns=['Property Type'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['Property Type'], drop_first=True)

# Treinar o modelo de regressão linear novamente
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Prever valores no conjunto de teste e calcular MSE
y_pred = linear_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print("MSE:", mse)

MSE: 2291456.1900715767


Treinando esse mesmo modelo com validação cruzada pelo risco de overfitting

In [256]:
from sklearn.model_selection import KFold, cross_val_score

# Função para calcular o MSE usando validação cruzada
def cross_val_mse(model, X, y, n_folds=5):
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    mse_scores = -cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
    return mse_scores.mean()

# Aplicar Target Encoding dentro da validação cruzada
kf = KFold(n_splits=5, shuffle=True, random_state=42)
mse_scores = []

for train_index, test_index in kf.split(X):
    X_train_fold, X_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    
    # Calcular médias da variável resposta para cada categoria de "Region" no conjunto de treino do fold
    region_target_mean = y_train_fold.groupby(X_train_fold['Region']).mean()
    
    # Substituir cada categoria em "Region" pela média correspondente nos conjuntos de treino e teste do fold
    X_train_fold.loc[:, 'Region'] = X_train_fold['Region'].map(region_target_mean)
    X_test_fold.loc[:, 'Region'] = X_test_fold['Region'].map(region_target_mean)
    
    # Preencher valores NaN no conjunto de teste com a média global do conjunto de treino
    X_test_fold['Region'].fillna(y_train_fold.mean(), inplace=True)
    
    # Aplicar codificação one-hot à coluna "Property Type"
    X_train_fold = pd.get_dummies(X_train_fold, columns=['Property Type'], drop_first=True)
    X_test_fold = pd.get_dummies(X_test_fold, columns=['Property Type'], drop_first=True)
    
    # Treinar o modelo e calcular MSE
    model = LinearRegression()
    model.fit(X_train_fold, y_train_fold)
    y_pred_fold = model.predict(X_test_fold)
    mse_fold = mean_squared_error(y_test_fold, y_pred_fold)
    mse_scores.append(mse_fold)


# Calcular a média dos MSEs obtidos nos folds
average_mse = sum(mse_scores) / len(mse_scores)
average_mse

C:\Users\Cliente\AppData\Local\Temp\ipykernel_18128\3522411572.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_fold.loc[:, 'Region'] = X_train_fold['Region'].map(region_target_mean)
C:\Users\Cliente\AppData\Local\Temp\ipykernel_18128\3522411572.py:21: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X_train_fold.loc[:, 'Region'] = X_train_fold['Region'].map(region_target_mean)
C:\Users\Cliente\AppData\Local\Temp\ipykernel_18128\3522411572.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

2136019.2173908777